In [ ]:
# drive mount 
from google.colab import drive 
drive.mount("/content/Mydrive/")

In [ ]:
# 현재 디렉토리 확인 
import os 
os.getcwd()

In [ ]:
# 경로 변경 
os.chdir('/content/Mydrive/MyDrive/Project III/네이버웹소설_크롤링')
os.getcwd()

In [ ]:
# 패키지 설치 
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

In [ ]:
# colab에서 webdriver 사용하기 위한 준비 
from selenium import webdriver
import time
from urllib.request import urlretrieve

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
#  웹 자원 로드 위해 3초 기다리기
driver.implicitly_wait(3)

# 특정 url 열기
driver.get("<크롤링할 웹소설 장르 페이지>")
time.sleep(3)
# 팝업창 닫기
popup =  driver.find_element_by_class_name("lk_finish")
popup.click()

# 웹소설 제목 특수문자 제거용
characters = '?!.,/' 

# 데이터 담을 리스트 
titles, writers, illustrators, stars, downloads, publishs, genres, hearts, img_paths = [], [], [], [], [], [], [], [], []
count = 0 # 파일명 생성 시 사용 

# 다음 페이지 클릭 
for j in range(0,13) : 
    start = time.time()
    print(j,"페이지")
    if j > 7 : 
        j = 7 
    if j >= 1 : 
        next_page = driver.find_element_by_xpath(f'//*[@id="content"]/div[2]/div[2]/a[{j}]')
        next_page.click()
        time.sleep(1.5)

    # 페이지 내 작품 클릭 
    for i in range(1,21) : 
        print(i,'번째 작품')
        driver.find_element_by_xpath(f'//*[@id="content"]/div[2]/ul/li[{i}]/a').click()
        time.sleep(1.5)

        # 제목 
        try : 
            title = driver.find_element_by_class_name("book_title").text
        except: 
            title = ''

        # 글작가
        try : 
            writer = driver.find_element_by_xpath('//*[@id="content"]/p/a[1]').text
        except: 
            writer = ''

        # 삽화가 
        try :
            illustrator = driver.find_element_by_xpath('//*[@id="content"]/p/a[2]').text
        except: 
            illustrator = ''

        time.sleep(1)

        # 별점 
        try :
            star = driver.find_element_by_xpath('//*[@id="content"]/div[1]/p[1]/em').text 
        except: 
            star = ''

        # 다운로드 
        try :
            download = driver.find_element_by_class_name("download").text
        except: 
            download = ''

        # 연재 날짜 
        try :
            publish = driver.find_element_by_class_name("publish").text
        except: 
            publish = ''

        # time.sleep(1)

        # 장르 
        try :
            genre = driver.find_element_by_class_name("genre").text
        except: 
            genre = ''

        # 하트 수 
        try :
            heart = driver.find_element_by_xpath('//*[@id="content"]/ul/li[1]/div/a/em').text 
        except: 
            heart = ''

        # 이미지 저장 
        try :
            image = driver.find_element_by_xpath('//*[@id="content"]/div[1]/a[1]/img')
            img_url = image.get_attribute("src")
            title = ''.join( x for x in title if x not in characters)
            img_path = f"./img_data/{count}.{title}.jpg"
            urlretrieve(img_url,img_path )
        except: 
            img_path = ''
        time.sleep(1)

        count += 1 
        titles.append(title)
        writers.append(writer)
        illustrators.append(illustrator)
        stars.append(star)
        downloads.append(download)
        publishs.append(publish)
        genres.append(genre)
        hearts.append(heart)
        img_paths.append(img_path)

        # 뒤로가기 
        driver.back()
    print("페이지 당: ", time.time()-start)

# 드라이버 닫기 
driver.quit()

In [ ]:
# 데이터 프레임 생성 
import pandas as pd 

df = pd.DataFrame({
    '제목' : titles, 
    '작가' : writers,
    '삽화가' : illustrators,
    '별점' : stars, 
    '다운로드수' : downloads, 
    '연재일' : publishs,
    '장르' : genres,
    '하트수' : hearts,
    '이미지주소' : img_paths
    
})

df

In [ ]:
# csv 저장 
df.to_csv("<저장경로>", index= False)